# Weighted Association Rules: Luật Kết Hợp Có Trọng Số

## Mục tiêu
- Mở rộng pipeline Apriori/FP-Growth với **weighted association rules**
- Tập trung vào **luật "niche"**: support thấp nhưng weighted support cao
- Phân tích insight kinh doanh từ các luật này

## Lý thuyết
Weighted support: Tỷ lệ giá trị hóa đơn chứa itemset trên tổng giá trị toàn bộ hóa đơn
- Ưu điểm: Phát hiện luật quan trọng trong hóa đơn giá trị cao dù xuất hiện ít

In [ ]:
# PARAMETERS

# Đường dẫn dữ liệu
CLEANED_DATA_PATH = "../data/processed/cleaned_uk_data.csv"
BASKET_BOOL_PATH = "../data/processed/basket_bool.pkl"  # Sử dụng pickle để giữ index

# Tham số khai thác
MIN_SUPPORT = 0.01  
MAX_LEN = 3
METRIC = "lift"
MIN_THRESHOLD = 1.0

# Lọc luật
FILTER_MIN_CONF = 0.3
FILTER_MIN_LIFT = 1.2

# Niche criteria
MAX_SUPPORT = 0.02  # Support thấp
MIN_WEIGHTED_SUPPORT = 0.05  # Weighted support cao

In [20]:
# Import libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import networkx as nx

%load_ext autoreload
%autoreload 2

# Add src to path
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("..")
else:
    project_root = cwd
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

from apriori_library import AssociationRulesMiner, FPGrowthMiner, DataVisualizer

# Set style
plt.style.use('default')
sns.set_palette("husl")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Load data
print("Loading cleaned data...")
df_cleaned = pd.read_csv(CLEANED_DATA_PATH)
print(f"Cleaned data shape: {df_cleaned.shape}")

print("Loading basket boolean...")
basket_bool = pd.read_pickle(BASKET_BOOL_PATH)
print(f"Basket bool shape: {basket_bool.shape}")
print(f"Index type: {basket_bool.index.dtype}")

# Check InvoiceValue
print(f"Sample InvoiceValue: {df_cleaned['InvoiceValue'].head()}")
print(f"Total value: £{df_cleaned['InvoiceValue'].sum():,.2f}")

Loading cleaned data...


C:\Users\Admin\AppData\Local\Temp\ipykernel_19280\1091762469.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cleaned = pd.read_csv(CLEANED_DATA_PATH)


Cleaned data shape: (485123, 12)
Loading basket boolean...
Basket bool shape: (18021, 4007)
Index type: object
Sample InvoiceValue: 0    139.12
1    139.12
2    139.12
3    139.12
4    139.12
Name: InvoiceValue, dtype: float64
Total value: £654,275,742.38


In [22]:
# Mine association rules
print("Mining frequent itemsets with FP-Growth...")
miner = FPGrowthMiner(basket_bool)
miner.mine_frequent_itemsets(min_support=MIN_SUPPORT, max_len=MAX_LEN)

print("Generating rules...")
miner.generate_rules(metric=METRIC, min_threshold=MIN_THRESHOLD)

print("Adding readable strings...")
miner.add_readable_rule_str()

print(f"Total rules before filtering: {len(miner.rules)}")

# Filter rules
filtered_rules = miner.filter_rules(
    min_confidence=FILTER_MIN_CONF,
    min_lift=FILTER_MIN_LIFT
)

print(f"Rules after filtering: {len(filtered_rules)}")

Mining frequent itemsets with FP-Growth...
Generating rules...
Adding readable strings...
Total rules before filtering: 3856
Rules after filtering: 2008


In [24]:
# Add weighted metrics
print("Adding weighted metrics...")
if miner.rules is None:
    raise ValueError("Rules not generated.")

# Dict invoice to InvoiceValue
invoice_values = df_cleaned.groupby('InvoiceNo')['InvoiceValue'].first()

total_weight = invoice_values.sum()

def calc_weighted_support(itemset):
    # Find invoices containing all items in itemset
    mask = basket_bool[list(itemset)].all(axis=1)
    invoices = basket_bool.index[mask]
    return invoice_values.loc[invoices].sum() / total_weight

rules = miner.rules.copy()
rules['weighted_support_antecedents'] = rules['antecedents'].apply(calc_weighted_support)
rules['weighted_support_consequents'] = rules['consequents'].apply(calc_weighted_support)
rules['weighted_support'] = rules.apply(lambda row: calc_weighted_support(row['antecedents'].union(row['consequents'])), axis=1)
rules['weighted_confidence'] = rules['weighted_support'] / rules['weighted_support_antecedents']
rules['weighted_lift'] = rules['weighted_confidence'] / rules['weighted_support_consequents']

miner.rules = rules

rules = miner.rules
print("Sample rules with weighted metrics:")
rules[['rule_str', 'support', 'weighted_support', 'confidence', 'weighted_confidence', 'lift', 'weighted_lift']].head()

Adding weighted metrics...
Sample rules with weighted metrics:


,rule_str,support,weighted_support,confidence,weighted_confidence,lift,weighted_lift
2601,"HERB MARKER PARSLEY, HERB MARKER ROSEMARY → HE...",0.010932,0.026342,0.951691,0.937082,74.567045,30.095955
2604,"HERB MARKER THYME → HERB MARKER PARSLEY, HERB ...",0.010932,0.026342,0.856522,0.846027,74.567045,30.095955
2982,"HERB MARKER MINT, HERB MARKER THYME → HERB MAR...",0.010599,0.024000,0.955000,0.955680,74.502403,28.373970
2987,"HERB MARKER ROSEMARY → HERB MARKER MINT, HERB ...",0.010599,0.024000,0.826840,0.712544,74.502403,28.373970
3222,"HERB MARKER MINT, HERB MARKER THYME → HERB MAR...",0.010432,0.023078,0.940000,0.919000,74.297105,29.495045


In [25]:
# Filter niche rules
print("Filtering niche rules...")
niche_rules = rules[
    (rules['support'] <= MAX_SUPPORT) & 
    (rules['weighted_support'] >= MIN_WEIGHTED_SUPPORT)
].sort_values('weighted_lift', ascending=False)

print(f"Found {len(niche_rules)} niche rules")
print("Top niche rules:")
niche_rules[['rule_str', 'support', 'weighted_support', 'lift', 'weighted_lift']].head(10)

Filtering niche rules...
Found 1326 niche rules
Top niche rules:


,rule_str,support,weighted_support,lift,weighted_lift
602,REGENCY TEA PLATE GREEN → REGENCY TEA PLATE R...,0.015593,0.056264,39.556782,11.905552
603,REGENCY TEA PLATE ROSES → REGENCY TEA PLATE G...,0.015593,0.056264,39.556782,11.905552
278,SMALL MARSHMALLOWS PINK BOWL → SMALL DOLLY MIX...,0.018589,0.054832,27.528659,11.493749
279,SMALL DOLLY MIX DESIGN ORANGE BOWL → SMALL MAR...,0.018589,0.054832,27.528659,11.493749
2114,FLORAL FOLK STATIONERY SET → MODERN FLORAL STA...,0.011487,0.062183,26.197729,9.137467
2115,MODERN FLORAL STATIONERY SET → FLORAL FOLK STA...,0.011487,0.062183,26.197729,9.137467
376,JUMBO BAG 50'S CHRISTMAS → JUMBO BAG VINTAGE ...,0.017369,0.061770,18.366980,9.101309
377,JUMBO BAG VINTAGE CHRISTMAS → JUMBO BAG 50'S ...,0.017369,0.061770,18.366980,9.101309
902,CHRISTMAS CRAFT LITTLE FRIENDS → CHRISTMAS CRA...,0.014428,0.061000,21.627363,9.079428
903,CHRISTMAS CRAFT TREE TOP ANGEL → CHRISTMAS CRA...,0.014428,0.061000,21.627363,9.079428


In [26]:
# Visualization
if len(niche_rules) > 0:
    # Scatter plot: support vs weighted_support
    fig = px.scatter(
        niche_rules, 
        x='support', 
        y='weighted_support',
        size='weighted_lift',
        color='confidence',
        hover_data=['rule_str'],
        title='Niche Rules: Low Support but High Weighted Support'
    )
    fig.show()
    
    # Bar chart top weighted_lift
    top_niche = niche_rules.head(10)
    fig2 = px.bar(
        top_niche, 
        x='rule_str', 
        y='weighted_lift',
        title='Top 10 Niche Rules by Weighted Lift'
    )
    fig2.show()
else:
    print("No niche rules found with current criteria")

# Insights Kinh Doanh từ Luật Niche

Dựa trên các luật có support thấp nhưng weighted support cao, rút ra các insight:

1. **Luật 1**: [Ví dụ] "RED RETROSPOT CHARLOTTE BAG → RED RETROSPOT PICNIC BAG"
   - Support thấp (ít khách mua cùng) nhưng weighted support cao
   - Ý nghĩa: Sản phẩm này thường được mua trong các đơn hàng lớn
   - Đề xuất: Quảng bá combo cho khách VIP, tạo gói quà tặng premium

2. **Luật 2**: [Ví dụ] "WOODEN FRAME ANTIQUE WHITE → WOODEN PICTURE FRAME WHITE FINISH"
   - Weighted lift cao cho thấy mối liên kết mạnh trong đơn giá trị
   - Đề xuất: Cross-selling trong catalog online cho khách hàng cao cấp

3. **Luật 3**: [Ví dụ] "VINTAGE UNION JACK BUNTING → VINTAGE UNION JACK MEMOBOARD"
   - Niche market: Khách hàng mua quà tặng theo chủ đề
   - Đề xuất: Tạo section "Gift Sets" với discount cho combo

4. **Luật 4**: [Ví dụ] "HOT WATER BOTTLE KEEP CALM → CHOCOLATE HOT WATER BOTTLE"
   - Sản phẩm seasonal hoặc promotional
   - Đề xuất: Push notification cho khách đã mua một trong hai

5. **Luật 5**: [Ví dụ] "REGENCY CAKESTAND 3 TIER → ROSES REGENCY TEACUP AND SAUCER"
   - Luxury dining set
   - Đề xuất: Chương trình loyalty cho khách mua full set

# 5.3.1.5 High-Utility Itemset Mining (HUIM) - Phiên bản nâng cao

## Lý thuyết HUIM
High-Utility Itemset Mining khác với Frequent Itemset Mining:
- **Frequent**: Tối ưu theo số lần xuất hiện (support).
- **Weighted (trong project này)**: Tối ưu theo tổng giá trị hóa đơn chứa itemset.
- **High-Utility**: Tối ưu theo "utility" (lợi nhuận/lợi ích) của từng item trong transaction.

Ví dụ: Một item rẻ tiền nhưng bán nhiều có thể không "high-utility" bằng item đắt tiền bán ít.

## Demo HUIM đơn giản
Trong demo này, giả sử utility của mỗi item là UnitPrice (giá bán).
Utility của itemset là tổng utility của items trong transaction chứa itemset.

Đây là version đơn giản, không dùng thuật toán HUIM chuyên nghiệp như UP-Growth.

### Code demo:

In [28]:
# Demo HUIM đơn giản
print("Demo High-Utility Itemset Mining...")

# Giả sử utility của item là UnitPrice trung bình
item_utilities = df_cleaned.groupby('Description')['UnitPrice'].mean().to_dict()

# Tính utility cho mỗi transaction
df_huim = df_cleaned.copy()
df_huim['item_utility'] = df_huim['Description'].map(item_utilities)
df_huim['transaction_utility'] = df_huim.groupby('InvoiceNo')['item_utility'].transform('sum')

# Tạo basket với utility
basket_utility = df_huim.groupby(['InvoiceNo', 'Description'])['item_utility'].sum().unstack().fillna(0)

# Chuyển thành boolean (có mặt)
basket_bool_huim = (basket_utility > 0).astype(bool)

# Tính utility của itemset (tổng utility của transactions chứa itemset)
def calc_utility_support(itemset):
    mask = basket_bool_huim[list(itemset)].all(axis=1)
    invoices = basket_bool_huim.index[mask]
    return df_huim[df_huim['InvoiceNo'].isin(invoices)]['transaction_utility'].sum()

# Demo với top frequent itemsets từ FP-Growth
frequent_itemsets = miner.frequent_itemsets.head(10)  # Top 10 frequent itemsets

huim_results = []
for idx, row in frequent_itemsets.iterrows():
    itemset = list(row['itemsets'])
    utility = calc_utility_support(itemset)
    huim_results.append({
        'itemset': itemset,
        'support': row['support'],
        'utility': utility
    })

huim_df = pd.DataFrame(huim_results)
huim_df['utility_per_support'] = huim_df['utility'] / huim_df['support']

print("Top itemsets by Utility:")
huim_df_sorted = huim_df.sort_values('utility', ascending=False).head()
print(huim_df_sorted)

# Visualize
fig = px.scatter(
    huim_df, 
    x='support', 
    y='utility',
    size='utility_per_support',
    title='High-Utility Itemsets: Support vs Utility'
)
fig.show()

Demo High-Utility Itemset Mining...
Top itemsets by Utility:
                                itemset   support       utility  \
1             [JUMBO BAG RED RETROSPOT]  0.107375  1.229755e+08   
0  [WHITE HANGING HEART T-LIGHT HOLDER]  0.119971  1.123605e+08   
7     [NATURAL SLATE HEART CHALKBOARD ]  0.067643  1.112713e+08   
2            [REGENCY CAKESTAND 3 TIER]  0.093502  1.026183e+08   
4             [LUNCH BAG RED RETROSPOT]  0.077243  9.595848e+07   

   utility_per_support  
1         1.145293e+09  
0         9.365625e+08  
7         1.644971e+09  
2         1.097498e+09  
4         1.242290e+09  


## Giải thích kết quả HUIM
- **Utility**: Tổng "lợi ích" (dựa trên UnitPrice) của transactions chứa itemset.
- **Sự khác biệt với Frequent/Weighted**:
  - Frequent: Chỉ quan tâm số lần xuất hiện.
  - Weighted: Quan tâm tổng giá trị hóa đơn.
  - HUIM: Quan tâm utility của từng item, có thể ưu tiên items đắt tiền dù ít xuất hiện.
- **Ví dụ**: "NATURAL SLATE HEART CHALKBOARD" có utility cao (1.64e9 per support), cho thấy giá trị kinh doanh lớn dù support thấp.

Đây là demo đơn giản. Để HUIM thực sự, cần thuật toán chuyên nghiệp như UP-Growth hoặc sử dụng SPMF library.